In [1]:
import glob
import numpy as np
import pandas as pd
from project_utils import *

%matplotlib inline

In [2]:
instruments = {s: i for i, s in enumerate([
    'violin',
    'clarinet',
    'saxphone',
    'bassoon',
])}

instrument = 'clarinet'

dirs = list(sorted(glob.glob('./data/Bach10/??-*')))
mid_files = ['{}/{}.mid'.format(x, x.split('/')[-1]) for x in dirs]
wav_files = ['{}/{}-{}.wav'.format(x, x.split('/')[-1], instrument) for x in dirs]
asl_files = ['{}/{}.asl'.format(x, x.split('/')[-1]) for x in dirs]

alignments = []
features = []
scores = []
for i in range(10):        
    alignments.append(pd.read_csv(asl_files[i], sep='\t', header=None, index_col=None))     
    scores.append(parse_midi(mid_files[i], instruments[instrument]))
    features.append(wav_features(wav_files[i]))
    features[-1] = features[-1][:alignments[-1].shape[0]]
    ids = alignments[-1][2] > 0.0
    alignments[-1] = alignments[-1][ids]
    alignments[-1][2] -= alignments[-1][2].min() - np.modf(alignments[-1][2].min())[0]
    features[-1] = features[-1][ids]
    alignments[-1] = alignments[-1].iloc[::4]
    features[-1] = features[-1][::4]
    
    scores[-1][1] -= scores[-1][1][0]    

In [3]:
GTs, Bs, SYs, K = prepare(scores, alignments)

In [4]:
K

18

In [5]:
np.max(alignments[0][2]), np.max(scores[0][1][:-1])

(31.948999999999998, 31.0)

In [6]:
GTs[0].shape

(601, 26)

In [7]:
SYs[0]

array([ 3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  0,  0,  0,  0,  0,  0,  0,
       16, 16, 16, 16, 16, 16, 16, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  3

In [8]:
Bs[0]

array([ 3,  2, 16,  0, 16, 14, 16,  0,  3,  2,  3,  2,  3,  2, 16,  3,  1,
        3,  2,  3, 16, 14,  5,  3,  1,  0], dtype=int32)